# <div style="font-family: Trebuchet MS; background-color: #000099; color: #FFFFFF; padding: 12px; line-height: 1;">Notebook Structure </div>

0. **Analysis Summary**
1. **Std Lib Imports**
2. **Data Import**- Covers basic data import and concat operation
3. **Building understanding of data**- Most of the heavy lifting on work here
    * Understanding: Knowing what's in the data before cleaning
    * Data cleaning: Preparing cleaned data (df_) where each row represent a row for each session or joint session of NA
4. **Secondary Import**- Imported data from wikipedia where we've merged given constituencies name with their city & areas for better EDA
5. **Exploration**- EDA of final data (df_final) and comments over relationships


# <div style="font-family: Trebuchet MS; background-color: #000099; color: #FFFFFF; padding: 12px; line-height: 1;">Analysis Summary </div>

**Prompt**: The dataset used is posted with question attached, "Find The Performance of Your MNA"

Hence, we've plotted the presense of MNA of respective local 'areas' by bar plot at the end this notebook. For example, if you were live in Karachi Central I, then MNA has attended most session across all constituencies in Sindh (Perhaps because Karachi Central I is perhaps one of the largest and most dense area in Sindh- maybe even in Pakistan)

**Results**: Here's how the top & worse performance constituencies have been based on their presence in NA sessions:

Sindh-
    Best: Karachi & Tharkparker Areas
    Worse: Matiari & Ghotki

Punjab-
    Best: Lahore-III, Muzzaffargarh IV, Gujranwala
    Worse: Faisalabad-I, Mianwali, Layyah, Gujrat

Balochistant
    Best: Mastung, Jafarabad
    Worse: Kech, Khuzdar

KPK
    Best: Chirtal, Manserah
    Worse: Mardan, South Waziristan, Islamabad III, Peshawar I

Simple google search of constituency name can tell you what area it covers:
A rather detailed view is shared and attached here: https://en.wikipedia.org/wiki/List_of_constituencies_of_Pakistan

**Caveats**
I've not viewed data in aggregate of multiple session sittings & dates. Perhaps, there could some value in going down to that depth to get some more insights. For future contributors, this could be a guideline.

# <div style="font-family: Trebuchet MS; background-color: #000099; color: #FFFFFF; padding: 12px; line-height: 1;">Std Lib Imports </div>

In [14]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 5000)
import re
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly.express as px

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [15]:
!pip install xlrd

In [16]:
!pip install openpyxl

**Methods** Self-defined

In [17]:
#Custom Method used
def session(col):
    return col[0]

def col_details(df, col_index):

    total= len(df[df.columns[col_index]])
    na_count= df[df.columns[col_index]].isna().sum()
    share_na_count= round(float(na_count/total)*100,2)

    print('Column:  ', df.columns[col_index])
    print('\nTotal Values', total, ' - NA count ', na_count, '- Share of NA ', share_na_count)
    print('\nValue_counts:')
    print(df[df.columns[col_index]].value_counts())

# <div style="font-family: Trebuchet MS; background-color: #000099; color: #FFFFFF; padding: 12px; line-height: 1;">Data Import </div>
1. Column views
2. Concat df
3. Sum of null values

In [18]:
root='data/Attendence of Members - Sessions 21 - 43.xls'

df_1= pd.read_excel(root, sheet_name= 'Session 1 to 20')
df_2= pd.read_excel(root,sheet_name= 'Attendence of Members - Session')

df_1.columns= ['meta', 'constituency','name','presense']
df_2.columns= ['meta', 'constituency','name','presense']

print('df_1 columns: ', df_1.columns)
print('\ndf_2 columns: ', df_2.columns)
print('\nlen df_1', len(df_1), ' len df_2', len(df_2))

df= df_1.append(df_2)

df.reset_index(inplace=True,drop=True)


cols=df.columns
print('********info**********:')
print(df.info())
print('\n **********isNa***********: ')
print(df.isna().sum())

df_1 columns:  Index(['meta', 'constituency', 'name', 'presense'], dtype='object')

df_2 columns:  Index(['meta', 'constituency', 'name', 'presense'], dtype='object')

len df_1 37247  len df_2 38992
********info**********:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76239 entries, 0 to 76238
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   meta          75618 non-null  object
 1   constituency  74888 non-null  object
 2   name          75844 non-null  object
 3   presense      74560 non-null  object
dtypes: object(4)
memory usage: 2.3+ MB
None

 **********isNa***********: 
meta             621
constituency    1351
name             395
presense        1679
dtype: int64


# <div style="font-family: Trebuchet MS; background-color: #000099; color: #FFFFFF; padding: 12px; line-height: 1;">Bulding understanding of data </div>

Imported as two dataframes (df_1, df_2), each df has a sub-composition if explored in below code:
1. NA has series of "Joint Sessions" and simple "Sessions": The attendence is recorded against each session of each type. Hence, it's critical two clean the data first with this view **before making any drops of null values**
2. Upon exploring the dataset, one can observe pattern of colums [Sl. no., constituency, Name of Member, Status] being recurrent columns for each session
3. Each constituency refers to particular area; I'm adding a secondary data set to merge for locations of constituencies for a better EDA (Explore more here: https://en.wikipedia.org/wiki/List_of_constituencies_of_Pakistan)

In [19]:
df[37040:37255]

,meta,constituency,name,presense
37040,255,Reserved Seat,Mr. Ramesh Kumar Vankwani,P
37041,256,Reserved Seat,Mr. Jamshed Thomas,P
37042,257,Reserved Seat,Dr. Darshan,P
37043,258,Reserved Seat,Mr. Kesoo Mal Kheeal Das,P
37044,259,Reserved Seat,Mr. Naveed Aamir Jeeva,P
37045,NaN,NaN,NATIONAL ASSEMBLY SECRETARIAT,NaN
37046,NaN,NaN,(Notice Office),NaN
37047,20th Session,NaN,"4th Sitting held on Friday, the 13th March, 2020",NaN
37048,The following Members National Assembly of Pak...,NaN,NaN,NaN
37049,Sl. No.,Constituency,Name of Member,Status


In [20]:
df.loc[(df['presense'] != 'P') & (df['presense'] != 'Status') &
       (~pd.isna(df['presense'])) & (df['presense'] != 'Attendance')]

,meta,constituency,name,presense
57019,32,NA-43,Mr. Noor-ul-Haq Qadri,A
72108,136,NA-192,Sardar Muhammad Khan Laghari,A


# **Data Cleaning**

Before applying any null drops, let's first segregate the sessions & joint sessions took place against each-
Approach:
1. We've first created new cols named 'session' & 'joint_session_number' against each occurance from cols[0] text
2. Next, we're simply broadcasting the session numbers over their applicable range of data
3. Lastly, dropping completely unnecessary rows

**End result:** Our df has session & joint_session number populated against each record

In [21]:
df['session']= df[df[cols[0]].str.contains('(\d)[a-z]+ Session|session', na=False, regex=True)][cols[0]].str.findall('(\d+)').apply(session)
df['joint_session_number']=df[df[cols[0]].str.contains('(\d)[a-z]+ joint|Joint', na=False, regex=True)][cols[0]].str.findall('(\d+)').apply(session)


C:\Users\HP\AppData\Local\Temp/ipykernel_20728/1797757456.py:1: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  df['session']= df[df[cols[0]].str.contains('(\d)[a-z]+ Session|session', na=False, regex=True)][cols[0]].str.findall('(\d+)').apply(session)
C:\Users\HP\AppData\Local\Temp/ipykernel_20728/1797757456.py:2: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  df['joint_session_number']=df[df[cols[0]].str.contains('(\d)[a-z]+ joint|Joint', na=False, regex=True)][cols[0]].str.findall('(\d+)').apply(session)


In [22]:
#Broadcasting
df_copy= df.copy()
df_copy_=df.copy()

num='NaN'
for i in df_copy.index:

    if str(df_copy.iloc[i,5])=='nan':
        if str(df_copy.iloc[i,4])=='nan':
            df_copy.iloc[i,5]=num

    if str(df_copy.iloc[i,5])!='nan':
        num= str(df_copy.iloc[i,5])
    else:
        num='NaN'
num='NaN'
for i in df_copy_.index:

    if str(df_copy_.iloc[i,4])=='nan':
        if str(df_copy_.iloc[i,5])=='nan':
            df_copy_.iloc[i,4]=num

    if str(df_copy_.iloc[i,4])!='nan':
        num= str(df_copy_.iloc[i,4])
    else:
        num='NaN'

In [23]:
df['joint_session_number'] = df_copy['joint_session_number']
df['session'] = df_copy_['session']

In [24]:
df_= df[~(
    (df[cols[2]].astype('str')=='nan') | #All members have their name populated; 'nan' represents metadata rows which we've extract already
    ((df[cols[2]].astype('str')=='Name of Member')) | #This is recurrent field occuring throughout df b/c multiple df being concated implicitly
    ((df[cols[3]].astype('str')=='nan')) |
    ((df[cols[3]].astype('str')=='Status'))
)]
df_.reset_index(drop=True, inplace=True)

#NaN session refers to no-applicable places
df_['session'].replace(to_replace='NaN', value='0',inplace=True)
df_['joint_session_number'].replace(to_replace='NaN', value='0',inplace=True)


C:\Users\HP\anaconda3\lib\site-packages\pandas\core\generic.py:6619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [25]:
#The constituency could be either contested (NA marked) or Reserved Seat as per the NA. Hence, creating a boolean here for later exploration
df_['is_contested']= np.where(
    df_[df_.columns[1]].str.contains("NA|Constituency", case=True)
    ,1,0)

C:\Users\HP\AppData\Local\Temp/ipykernel_20728/3471191169.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_['is_contested']= np.where(


In [26]:
#There are multiple spelling variations of reserved seats; creating a boolean out of it for ease
df_[df_.is_contested==0][cols[1]].value_counts()

Reserved Seat     16972
Reserve Seat        458
Reserved Seats      130
Reserved seat       101
Reserved Seate       64
Reserve seat         58
Name: constituency, dtype: int64

In [27]:
df_['is_reserved']= np.where(df_[cols[1]].str.contains("Reserv", case=True, regex=True),1,0)

C:\Users\HP\AppData\Local\Temp/ipykernel_20728/2100122556.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_['is_reserved']= np.where(df_[cols[1]].str.contains("Reserv", case=True, regex=True),1,0)


In [28]:
df_.columns= ['sr_no','constituency','name','presence','session_no', 'join_session_number','is_contested','is_reserved']

# # Cleaned data-

We've cleaned data of session 1-43 & Joint session 1-11 in "df_"

For places where session_no & join_session_number is 0, it means that record is insig for particular choice of session

# <div style="font-family: Trebuchet MS; background-color: #000099; color: #FFFFFF; padding: 12px; line-height: 1;">Secondary Data Import </div>
1. Importing dataset
2. Cleaning for areas of constituencies
3. Merging current data with last cleaned data for final version

In [29]:
df_3= pd.read_excel('../input/pk-constituencies-data/PK constituencies.xlsx')

FileNotFoundError: [Errno 2] No such file or directory: '../input/pk-constituencies-data/PK constituencies.xlsx'

In [ ]:
area=[]
for i in df_3.index:
    a=re.findall(r'NA-\d*|,|([a-zA-Z0-9- ]*)',df_3.iloc[i].Constituency)
    a.sort(reverse=True)
    area.append(a[0])

const=[]
for i in df_3.index:
    a=re.findall(r'NA-\d*',df_3.iloc[i].Constituency)
    a.sort(reverse=True)
    const.append(a[0])


In [ ]:
df_3['Area']=area
df_3['const']= const

In [ ]:
df_3.rename(columns={'City': 'province', 'Area':'area'}, inplace=True)
df_3.head()

In [ ]:
df_.head()

In [ ]:
df__= pd.merge(df_,df_3, how='left', left_on='constituency', right_on='const')

In [ ]:
df__.head()

In [ ]:
final_features= ['constituency','province','area','name','session_no','join_session_number','is_contested','is_reserved']
df_final= df__[final_features]

# <div style="font-family: Trebuchet MS; background-color: #666699; color: #FFFFFF; padding: 12px; line-height: 1.5;">Final Dataframe </div>

In [ ]:
df_final

# <div style="font-family: Trebuchet MS; background-color: #000099; color: #FFFFFF; padding: 12px; line-height: 1;">Exploration </div>
1. Session & Joint Session wise distribution of attendance
2. Top & Under represented areas in each city

In [ ]:
"""
Grouping for session wise cumilative strength
"""

session_province_attendance= df_final[
    (df_final.session_no!='0') #=0 are places with joint session
].groupby(['session_no','province']).count()['name'].reset_index()

session_province_attendance.session_no= session_province_attendance.session_no.astype(int)
session_province_attendance.sort_values('session_no').reset_index()


joint_session_province_attendance= df_final[
    (df_final.join_session_number!='0')
].groupby(['join_session_number','province']).count()['name'].reset_index()

joint_session_province_attendance.join_session_number= joint_session_province_attendance.join_session_number.astype(int)
joint_session_province_attendance.sort_values('join_session_number').reset_index()
joint_session_province_attendance.head()

In [ ]:
session_province_attendance.sort_values('session_no')
session_province_attendance.head()

In [ ]:
import plotly
plotly.offline.init_notebook_mode (connected = True)

In [ ]:
px.bar(session_province_attendance, y='session_no', x='name',color='province',orientation='h',
      labels={
                     "session_no": "Session Number",
                     "name": "Count of Members Present",
                 },
                title="Strength of presence in all sittings of a NA session (Post 2018)"
      )

In [ ]:
fig=px.bar(joint_session_province_attendance, y='join_session_number', x='name',color='province',orientation='h',
      labels={
                     "join_session_number": "Joint Session Number",
                     "name": "Count of Members Present",
                 },
                title="Strength of presence in all sittings of a NA joint session (Post 2018)"
      )
fig.show()

# <div style="font-family: Trebuchet MS; background-color: #666699; color: #FFFFFF; padding: 12px; line-height: 1.5;">Observations on above bar charts of session strength </div>

1. In normal sessions, there seems to be no significant trend over how the strength of presense is varying. A more operational context here could help (looking for your comments on this). For example- **Post 28 session, it was a period of Mar-22 where multiple members were refraining to be present in NA given the no-confidence motion against PK PM Imran Khan**

2. The variations of strength in normal session is much greater than much stable variation in joint session's strength

In [ ]:
df_final

In [ ]:
session_area_attendance= df_final[
    (df_final.session_no!='0')
].groupby(['province','area']).count()['name'].reset_index()

session_area_attendance.sort_values('name',ascending=False).reset_index().head()


joint_session_area_attendance= df_final[
    (df_final.join_session_number!='0')
].groupby(['province','area']).count()['name'].reset_index()


joint_session_area_attendance.sort_values('name', ascending=False).reset_index().head()

# <div style="font-family: Trebuchet MS; background-color: #666699; color: #FFFFFF; padding: 12px; line-height: 1.5;">Performance of you MNA </div>

If we assume that attendance of an MNA in the sessions of NA is an indicator of performance, then a distribution of how the presense looks like in all sessions post 2018 could be good view.
Although we could look the visual with MNA name, but for understanding, we're going to plot **Name of Contituency Area** (Ex: Karachi East I) and **Count of its MNA's presence** as bar chart.

Below, we've given province wise distribution sorted in desc order where first bar is the top performer (vice versa for last bar).

In [ ]:
plt.figure(figsize=(10,20))
plt.title("Sindh")
fig=sns.barplot(data=session_area_attendance[session_area_attendance.province=='Sindh'].sort_values('name', ascending=False),
    y="area", x="name")

fig.set(xlabel='Total count of presense', ylabel='Name of constituency')
fig;

In [ ]:
plt.figure(figsize=(10,40))
plt.title("Punjab")
fig=sns.barplot(data=session_area_attendance[session_area_attendance.province=='Punjab'].sort_values('name', ascending=False),
    y="area", x="name")


fig.set(xlabel='Total count of presense', ylabel='Name of constituency')
fig;

In [ ]:
plt.figure(figsize=(10,10))
plt.title("Balochistan")
fig=sns.barplot(data=session_area_attendance[session_area_attendance.province=='Balochistan'].sort_values('name', ascending=False),
    y="area", x="name")


fig.set(xlabel='Total count of presense', ylabel='Name of constituency')
fig;

In [ ]:
plt.figure(figsize=(10,20))
plt.title("KPK")
fig=sns.barplot(data=session_area_attendance[session_area_attendance.province=='KPK'].sort_values('name', ascending=False),
    y="area", x="name")


fig.set(xlabel='Total count of presense', ylabel='Name of constituency')
fig;

**Thanks**
